In [ ]:
import os
import json
from tqdm import tqdm

os.chdir('dataset/Diff_Quality_Estimation')


In [1]:
from ds import CodeDataset

In [2]:
from filenames import train_files

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
device = torch.device('cuda')
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
cpu = torch.device('cpu')

In [6]:
next(model.parameters()).device

device(type='cuda', index=0)

In [7]:
from torch.utils.data import DataLoader

dataset = CodeDataset(train_files[:1], '-testjs', tokenizer, 512, lambda x: x[0], lambda x: [1, 0] if x[1] else [0, 1])
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [7]:
batch = next(iter(dataloader))


In [8]:
import gc; gc.collect()

0

In [5]:
model.load_state_dict(torch.load('lang-classifier2.pt'))
optimizer.load_state_dict(torch.load('lang-optimizer2.pt'))

In [17]:
torch.cuda.empty_cache()


In [ ]:
model.train()
num_epochs = 10

for epoch in range(num_epochs):
    for filename in train_files:
        dataset = CodeDataset([filename], '-testjs', tokenizer, 512, lambda x: x[0], lambda x: [1, 0] if x[1] else [0, 1])
        dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
        
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
    
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()


 79%|███████▊  | 1636/2081 [49:38<14:20,  1.93s/it]

In [27]:
0

0

In [28]:
torch.save(model.state_dict(), 'lang-classifier2.pt')
torch.save(optimizer.state_dict(), 'lang-optimizer2.pt')

In [ ]:
from filenames import all_files
from torch.utils.data import Subset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

N = 160  # Number of items you want in your DataLoader

val_dataset = CodeDataset(train_files[0:1], '-testjs', tokenizer, 512, lambda x: x[0], lambda x: [1, 0] if x[1] else [0, 1])
limited_dataset = Subset(val_dataset, indices=range(N))
val_loader = DataLoader(limited_dataset, batch_size=16, shuffle=True)

def evaluate(model, val_loader):
    model.eval()
    predictions, true_labels, list_logits = [], [], []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
    
            outputs = model(input_ids, attention_mask=attention_mask)

            logits = outputs.logits
            print(input_ids, attention_mask)
            predictions.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(torch.argmax(labels, dim=1).tolist())
    
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy, precision, recall, f1, predictions, true_labels

# Example evaluation after training
accuracy, precision, recall, f1, predictions, true_labels = evaluate(model, val_loader)
print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}')


In [38]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [9]:
batch1 = next(iter(dataloader))
batch2 = next(iter(dataloader))
#model.classifier.out_proj.state_dict()

In [24]:
batch2['input_ids'].to(cpu)

tensor([[    0, 37815, 18851,  ...,     1,     1,     1],
        [    0,  4684, 22565,  ...,     1,     1,     1],
        [    0,  4684, 38650,  ...,  1437,  1437,     2],
        ...,
        [    0, 49453,   111,  ...,     1,     1,     1],
        [    0, 41975, 11988,  ...,     1,     1,     1],
        [    0, 37815, 18851,  ...,     4,  5330,     2]])

In [26]:
batch = batch2
model.to(cpu).roberta.embeddings(batch2['input_ids'].to(cpu))

tensor([[[-0.3412,  0.2680,  0.3021,  ..., -0.2898,  0.8585,  0.8295],
         [-0.1310,  0.5211,  1.1932,  ..., -2.2609,  0.9663,  0.4517],
         [-0.5610,  0.1916,  1.3016,  ..., -2.5255,  0.5054,  0.9198],
         ...,
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718]],

        [[-0.3412,  0.2680,  0.3021,  ..., -0.2898,  0.8585,  0.8295],
         [-0.5919,  0.4964,  1.3286,  ..., -1.3175,  0.9040,  0.4439],
         [-0.2158,  0.0730,  2.2302,  ..., -2.1365,  0.8708,  0.8628],
         ...,
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718]],

        [[-0.3412,  0.2680,  0.3021,  ..., -0.2898,  0.8585,  0.8295],
         [-0.5919,  0.4964,  1.3286,  ..., -1

In [ ]:
model.to(cpu).roberta(batch1['input_ids'].to(cpu), attention_mask = batch1['attention_mask'])

In [15]:
(batch['input_ids'].to(cpu), batch['attention_mask'].to(cpu))

(tensor([[    0, 37815, 18851,  ...,     1,     1,     1],
         [    0,  4684, 22565,  ...,     1,     1,     1],
         [    0,  4684, 38650,  ...,  1437,  1437,     2],
         ...,
         [    0, 49453,   111,  ...,     1,     1,     1],
         [    0, 41975, 11988,  ...,     1,     1,     1],
         [    0, 37815, 18851,  ...,     4,  5330,     2]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]))

In [20]:
batch = batch1
model.to(cpu).roberta.embeddings(batch['input_ids'].to(cpu))

tensor([[[-0.3412,  0.2680,  0.3021,  ..., -0.2898,  0.8585,  0.8295],
         [-0.6888,  0.1610,  2.2540,  ..., -1.1899,  0.9799,  0.0969],
         [-0.6100,  0.4691,  1.2676,  ..., -1.3877,  0.5095,  0.9338],
         ...,
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718]],

        [[-0.3412,  0.2680,  0.3021,  ..., -0.2898,  0.8585,  0.8295],
         [-0.1364,  0.2028,  2.1119,  ..., -2.2385,  0.9318,  0.1240],
         [-0.1204,  0.5067,  2.3155,  ..., -1.1716,  0.4899,  0.8391],
         ...,
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718],
         [-0.4277,  0.3331,  1.7303,  ..., -1.6854,  0.2706,  0.2718]],

        [[-0.3412,  0.2680,  0.3021,  ..., -0.2898,  0.8585,  0.8295],
         [-0.2499,  0.1800,  2.2900,  ..., -2